This notebook has 3 main steps:
- Step 1: Use ARC's API to conveniently run geometry optimizations and frequency calculations for ~400 reference species in preparation for BAC fitting
    - Recommended to comment out some lines in `ARC/arc/job/trsh.py` to ensure that ARC will only only troubleshoot by changing the optimization algorithm and not be changing the LoT or the software. For example, if using Gaussian to calculate BACs at some LoT, it would be wise to comment out these [lines](https://github.com/ReactionMechanismGenerator/ARC/blob/master/arc/job/trsh.py#L837-L844) and these [lines](https://github.com/ReactionMechanismGenerator/ARC/blob/master/arc/job/trsh.py#L852-L870).
- Step 2: Use Arkane to calculate H298 and add the values for the new LoT to our reference database
- Step 3: Finally, use Arkane to perform BAC fitting

Briefly, [ARC](https://github.com/ReactionMechanismGenerator/ARC) (Automated Rate Calculator) is a software developed by the RMG team for automating electronic structure calculations, which are then used to calculate thermochemical and kinetic parameters. Installation instructions can be found on ARC's documentation [link](https://reactionmechanismgenerator.github.io/ARC/installation.html).

In [ ]:
# Add RMG-Py and ARC to Python Path (Optional)
import sys
# this is an example path if running the notebook on supercloud
# replace <USERNAME> with your username
sys.path.append("/home/gridsan/<USERNAME>/RMG/RMG-Py/")
sys.path.append("/home/gridsan/<USERNAME>/RMG/ARC/")

In [ ]:
import os
import shutil

from arkane.encorr.reference import ReferenceDatabase
from arkane.ess import ess_factory
from rmgpy.molecule import Molecule

from arc import ARC
from arc.species import ARCSpecies
%matplotlib notebook

In [ ]:
# load the reference database
database = ReferenceDatabase()
database.load()

# Step 0) Define ARC Settings

## Step 0.1) Model chemistry and desired calculations

In [ ]:
######################################## USER INPUT ########################################
parent_folder = '/home/gridsan/<USERNAME>/ARC/Projects/'
project = 'BAC_wb97xd3_def2tzvp'

ess_settings = {'qchem': ['local']}
job_types = {'rotors': False,
             'conformers': False,
             'fine': False,
             'freq': True,
             'opt': True,
             'sp': True,
             }

max_job_time = 500  # hours

method = 'wb97x-d3'
basis = 'def2-tzvp'
software = 'qchem'
model_chemistry = f'{method}/{basis}'

# Set the Level of Theory
level_of_theory = {'method': method, 'basis': basis}

opt_level = level_of_theory
freq_level = level_of_theory
scan_level = level_of_theory
sp_level = level_of_theory
conformer_level = level_of_theory
ts_guess_level = level_of_theory
###################################### END USER INPUT ######################################
project_dir = os.path.join(parent_folder, project)

## Step 0.2) Define desired species to run (RUN ONLY ONE OF THE FOLLOWING CELLS)

### All reference species

In [ ]:
project_spcs = database.reference_sets['main']

### Species with specific indices

In [ ]:
indices = []
project_spcs = database.get_species_from_index(indices)

# Step 1) Use ARC API to run the QM jobs

## Step 1.1 Determine which species need to be run

In [ ]:
already_done = []
done_jobs = set()

# check for completed species by seeing which have opt, freq, and sp folders.
# if all folders are present, assume the species has completed its calculations.
# alternatively, could check output/status.yml to get the status of each species
for root, dirs, files in os.walk(os.path.join(project_dir, 'calcs', 'Species')):
    dirs.sort()
    for directory in dirs:
        for root1, dirs1, files1 in os.walk(os.path.join(root, directory)):
            if len(dirs1) == 3:
                done_jobs.add(directory)
            break
    break

already_done = list(done_jobs)

spcs_to_run = []
for ref_spcs in project_spcs:
    if f'spcs{ref_spcs.index}' not in already_done:
        spcs = ARCSpecies(label=f'spcs{ref_spcs.index}',
                          adjlist=ref_spcs.adjacency_list,
                          # default xyz is to start from the wb97mv conformer searching from Colin
                          xyz=ref_spcs.get_default_xyz(),
                          charge=ref_spcs.charge,
                          multiplicity=ref_spcs.multiplicity,
                          is_ts=ref_spcs.is_ts,
                          )
        spcs.mol = Molecule().from_adjacency_list(ref_spcs.adjacency_list, 
                                                  raise_atomtype_exception=False,
                                                  raise_charge_exception=False)
        spcs.multiplicity = ref_spcs.multiplicity
        spcs.charge = ref_spcs.charge
        
        spcs_to_run.append(spcs)

print(f'\nThe following {len(done_jobs)} species are already complete: {done_jobs}')
print(f'\nThe following {len(spcs_to_run)} species still need to be run: {[s.label for s in spcs_to_run]}')

In [ ]:
previous_job_paths = {}

## Step 1.2) Add path for opt/freq files (optional, if run in previous ARC job)
- Useful if doing composite LoT in which geometry optimization + frequency calculation are at a lower LoT and single point energy is at a higher LoT. Otherwise, can skip this cell

In [ ]:
path_to_logs = '/home/gridsan/<USERNAME>/PATH_TO_LOG_FILES/'

# Organize files by index
qm_logs = {}
for filename in os.listdir(path_to_logs):
    indx = filename.split('_')[0]
    qm_logs[indx] = os.path.join(path_to_logs, filename)
    
# Add files to restart dictionary
previous_job_paths = {}
for arc_spcs in spcs_to_run:
    label = arc_spcs.label
    indx = label[4:]
    previous_job_paths[label] = {'geo': qm_logs[indx], 'freq': qm_logs[indx]}

## Step 1.3) Run ARC

In [ ]:
arc_species_list = spcs_to_run

arc = ARC(project=project,
          species=arc_species_list,
          ess_settings=ess_settings,
          job_types=job_types,
          max_job_time=max_job_time,
          opt_level=opt_level,
          freq_level=freq_level,
          scan_level=scan_level,
          sp_level=sp_level,
          conformer_level=conformer_level,
          ts_guess_level=ts_guess_level,
          compare_to_rmg=False,
          calc_freq_factor=False,
          running_jobs=previous_job_paths,
          )

status_dict = arc.execute()

## Step 1.4) Reorganize the raw QM output from ARC folders to new opt+freq and sp folders

In [ ]:
os.mkdir('freq-folder')
os.mkdir('sp-folder')

When running ARC above, it may struggle with a handful of species by giving negative frequencies or non-converged geometry optimizations. Read the end of the ARC log file to find the few species that had negative frequencies (if applicable). Alternatively, this code could be modified to automatically parse the frequenies from the QM output files.

In [ ]:
path = os.path.join(parent_folder, project, 'calcs', 'Species')
no_freq = []  # these are the species that had convergence errors on opt so they never got to freq

for root, dirs, files in os.walk(path):
    # dirs is [spcs0, spcs1, ..spcs426]
    # there are a few places that skip an index so it's not perfectly sequentially
    dirs.sort()
    for directory in dirs:
        index = int(directory[4:])  # parse the index from spcs0 for example
        for root1, dirs1, files1 in os.walk(os.path.join(root, directory)):
            dirs1.sort()  # dirs1 should now be ['freq', 'opt', 'sp']
            
            # ensure no extra jobs have been run by checking there are exactly 3 folders
            try:
                assert len(dirs1) == 3
            except Exception as e:
                print(e)
                print(directory, dirs1)

            # copy the frequency output file
            freq_dir = dirs1[0]
            if 'freq' not in freq_dir:
                print(f'index {index} was missing a frequency job')
                no_freq.append(directory)
                break  # if no frequency job, then there will be no sp job either so just break out of this loop
            else:
                for root2, dirs2, files2 in os.walk(os.path.join(root1, freq_dir)):
                    files2.sort()
                    output = os.path.join(root2, 'output.out')
                    dst = os.path.join('freq-folder', str(index)+ '_output.out')
                    shutil.copy(output, dst)
                    break
            
            # copy the sp output file
            sp_dir = dirs1[2]
            if 'sp' not in sp_dir:
                print(f'index {index} was missing an sp job')
            else:
                for root2, dirs2, files2 in os.walk(os.path.join(root1, sp_dir)):
                    files2.sort()
                    output = os.path.join(root2, 'output.out')
                    dst = os.path.join('sp-folder', str(index)+ '_output.out')
                    shutil.copy(output, dst)
                    break
            break
    break
print(f'No frequencies: {no_freq}')

# Step 2) Use Arkane to calculate H298 

In [ ]:
from collections import defaultdict, Counter
import glob
import os
import re

from rdkit.Chem import GetPeriodicTable

from rmgpy.quantity import ScalarQuantity
from rmgpy.statmech import HarmonicOscillator, IdealGasTranslation, LinearRotor, NonlinearRotor
from rmgpy.thermo import ThermoData

from arkane.common import symbol_by_number
from arkane.encorr.corr import get_atom_correction, assign_frequency_scale_factor
from arkane.encorr.reference import CalculatedDataEntry, ReferenceDatabase
from arkane.modelchem import LevelOfTheory, CompositeLevelOfTheory

## Step 2.1) Define level of theory

In [ ]:
freq_lot = LevelOfTheory(method=method,
                         basis=basis,
                         software=software,
                         )
energy_lot = LevelOfTheory(method=method,
                           basis=basis,
                           software=software,
                           )
# could define composite LoT if applicable
# energy_lot = CompositeLevelOfTheory(freq=freq_lot, energy=energy_lot)

freq_log_dir = 'freq-folder'
energy_log_dir = 'sp-folder'

## Step 2.2) Define a function for using the Arkane API to calculate thermo

In [ ]:
def log_to_enthalpy(freq_lot, energy_lot, freq_log, energy_log=None, temp=298.15):
    freq_log = ess_factory(freq_log)
    energy_log = freq_log if energy_log is None else ess_factory(energy_log)
    
    conformer, _ = freq_log.load_conformer()
    # Perform quick checks
    assert conformer.spin_multiplicity > 0
    assert any(isinstance(mode, IdealGasTranslation) for mode in conformer.modes)
    assert any(isinstance(mode, (LinearRotor, NonlinearRotor)) for mode in conformer.modes)
    assert any(isinstance(mode, HarmonicOscillator) for mode in conformer.modes)
    
    coords, nums, masses = energy_log.load_geometry()
    assert len(nums) > 1
    
    atoms = Counter([symbol_by_number[int(n)] for n in nums]) 
    conformer.coordinates = (coords, 'angstroms')
    conformer.number = nums
    conformer.mass = (masses, 'amu')
    
    freq_scale_factor = assign_frequency_scale_factor(freq_lot)
    frequencies = conformer.modes[2].frequencies.value_si
    for mode in conformer.modes:
        if isinstance(mode, HarmonicOscillator):
            mode.frequencies = (frequencies * freq_scale_factor, "cm^-1")
    if freq_scale_factor == 1:
        print('WARNING: Frequency scale factor is 1')
    zpe_scale_factor = freq_scale_factor / 1.014
    
    # get electronic energy
    energy = energy_log.load_energy(zpe_scale_factor=zpe_scale_factor)  # J/mol
    
    # add ZPE
    energy += freq_log.load_zero_point_energy() * zpe_scale_factor if len(nums) > 1 else 0  # J/mol
    
    # add AECs
    energy += get_atom_correction(energy_lot, atoms)  # J/mol
    conformer.E0 = (energy / 4184, 'kcal/mol')
    
    return ScalarQuantity((conformer.get_enthalpy(temp) + conformer.E0.value_si) / 4184, 'kcal/mol')

periodic_table = GetPeriodicTable()

def log_to_xyz_dict(log):
    log = ess_factory(log)
    coords, nums, _ = log.load_geometry()
    syms = [symbol_by_number[int(n)] for n in nums]
    return {
        'coords': coords,
        'isotopes': [periodic_table.GetMostCommonIsotope(s) for s in syms],
        'symbols': syms
    }

## Step 2.3) Match all of the reference species to log files

In [ ]:
re_int = re.compile(r'[0-9]+')
ref_spcs = {spc.index: spc for spc in database.reference_sets['main']}

# Each log file must start with the index followed by an underscore
def get_logs(log_dir):
    if log_dir is None:
        return defaultdict(lambda: None)
    
    logs = glob.glob(os.path.join(log_dir, '*.out'))
    logs.extend(glob.glob(os.path.join(log_dir, '*.log')))
    logs = {int(re_int.findall(os.path.basename(log))[0]): log for log in logs}
    
    num_missing = 0
    for missing_idx in ref_spcs.keys() - logs.keys():
        print(f'{ref_spcs[missing_idx]} is missing a log file in {log_dir}')
        num_missing += 1
    print(f'A total of {num_missing} reference species are missing a log file!\n')    
    
    return logs
    
freq_logs = get_logs(freq_log_dir)
energy_logs = get_logs(energy_log_dir)

In [ ]:
# sanity check
assert len(energy_logs) == len(freq_logs)

## Step 2.4) Compute thermo and update values for the reference species

In [ ]:
logs_iter = freq_logs if energy_log_dir is None else energy_logs
for idx in logs_iter.keys():
    # Ignore extra logs that we don't have ref species for
    if idx not in ref_spcs:
        continue
    print(idx)
    hf298 = log_to_enthalpy(freq_lot, energy_lot, freq_logs[idx], energy_log=energy_logs[idx])
    
    # Update thermo if already in calculated data
    if energy_lot in ref_spcs[idx].calculated_data:  
        ref_spcs[idx].calculated_data[energy_lot].thermo_data.H298 = hf298
    # Make new calculated data entry otherwise
    else:
        ref_spcs[idx].calculated_data[energy_lot] = CalculatedDataEntry(ThermoData(H298=hf298), xyz_dict=log_to_xyz_dict(freq_logs[idx]))

## Step 2.5) Save the new enthalpy values to RMG-database


In [ ]:
database.save()

# Step 3) Create Arkane input file for BAC fitting

In [ ]:
bac_input = f"""lot = LevelOfTheory(
    method='{method}',
    basis='{basis}',
    software='{software}'
)
kwargs = {{
    'weighted': False,
    'write_to_database': True,
    'overwrite': True,
}}

# Petersson-type
bac(
    level_of_theory=lot,
    bac_type='p',  # Petersson
    **kwargs
)

# Melius-type
bac(
    level_of_theory=lot,
    bac_type='m',  # Melius
    fit_mol_corr=True,
    global_opt=True,
    global_opt_iter=5,
    **kwargs
)
"""

In [ ]:
with open("bac_arkane.py", "w") as f:
    f.write(bac_input)

Then do `python RMG-Py/Arkane.py bac_arkane.py`